In [2]:
import pongGym
import random
import numpy as np
import os
env = pongGym.DoublePong()
PAD_HEIGHT = 80
HALF_PAD_HEIGHT = PAD_HEIGHT // 2

pygame 2.1.0 (SDL 2.0.16, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [21]:
def encode(state):
    pd1y = state[21]
    pd2y = state[23]
    st = [(max(0, state[i*4+0] + state[i*4+2] * 4), abs(state[i*4+1] + state[i*4+3] * 4), state[i*4+2], state[i*4+3]) for i in range(5)]
    st.sort(key = lambda s: s[0])

    now = 1
    re = 0
    print(st)
    for i in range(5):
        ty = 0
        if(st[i][1] > pd1y + HALF_PAD_HEIGHT):
            ty = 1
        elif(st[i][1] < pd1y - HALF_PAD_HEIGHT):
            ty = 2
        print("0:", i , ty*now)
        re += now * ty
        now *= 3
        ty = 0
        if(st[i][1] > pd2y + HALF_PAD_HEIGHT):
            ty = 1
        elif(st[i][1] < pd2y - HALF_PAD_HEIGHT):
            ty = 2
        print("0:", i , ty*now)
        re += now * ty
        now *= 3
    return re
state = env.reset()

In [26]:
state = [558,141,5,-4,477,80,-2,-3,449,389,-2,-4,196,240,-4,0,280,256,3,-3,1,240,61,440]
print(state, len(state))
print(encode(state))

[558, 141, 5, -4, 477, 80, -2, -3, 449, 389, -2, -4, 196, 240, -4, 0, 280, 256, 3, -3, 1, 240, 61, 440] 24
[(180, 240, -4, 0), (292, 244, 3, -3), (441, 373, -2, -4), (469, 68, -2, -3), (578, 125, 5, -4)]
0: 0 0
0: 0 6
0: 1 0
0: 1 54
0: 2 81
0: 2 486
0: 3 1458
0: 3 4374
0: 4 13122
0: 4 39366
58947


In [3]:


q_table = np.zeros([(3*3)**5, env.action_space.n])
weight_path = "q_learning_{}.npy".format(8)
if os.path.isfile(weight_path):
    with open(weight_path, 'rb') as f:
        q_table = np.load(f)

In [4]:
print(q_table)

[[ 0.10595895  0.07077589  0.06184354  0.06426361]
 [-0.88543385 -1.02056652 -0.21745695 -0.69002132]
 [ 0.08235181 -0.88485404  0.0227449  -0.17054959]
 ...
 [ 0.17919906  0.19869572  0.03130001  0.19672003]
 [ 0.22799573  0.19742117  0.10831008  0.30209988]
 [-0.63263955 -0.78469977 -0.73861277 -0.03337042]]


In [ ]:
import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.2

# For plotting metrics
all_epochs = []
all_penalties = []
all_reward = 0

for i in range(1, 100001):
    state = env.reset()
    state = encode(state)
    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values
        next_state, reward, done, info = env.step(action)
        next_state = encode(next_state)
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1
        all_reward += reward
        state = next_state
        epochs += 1
        
    if i % 100 == 0:
#         clear_output(wait=True)
        print(f"Episode: {i}")
        print(f"average reward: {all_reward / 100}")
        all_reward = 0

print("Training finished.\n")

Episode: 100
average reward: -145.27
Episode: 200
average reward: -28.88
Episode: 300
average reward: -101.74
Episode: 400
average reward: -67.1
Episode: 500
average reward: -41.85
Episode: 600
average reward: -89.8
Episode: 700
average reward: -112.69
Episode: 800
average reward: -55.72
Episode: 900
average reward: -25.42
Episode: 1000
average reward: -36.74
Episode: 1100
average reward: -100.56
Episode: 1200
average reward: -75.54
Episode: 1300
average reward: -64.42
Episode: 1400
average reward: -43.09
Episode: 1500
average reward: -17.02
Episode: 1600
average reward: -32.55
Episode: 1700
average reward: -60.25
Episode: 1800
average reward: -56.35
Episode: 1900
average reward: -43.7
Episode: 2000
average reward: -56.94
Episode: 2100
average reward: -54.71
Episode: 2200
average reward: -15.36
Episode: 2300
average reward: -55.71
Episode: 2400
average reward: -67.47
Episode: 2500
average reward: -18.87
Episode: 2600
average reward: -47.53
Episode: 2700
average reward: 11.07
Episode: 2

In [ ]:
with open(weight_path, 'wb') as f:
    np.save(f, q_table)

In [ ]:
import time
total_epochs, total_penalties = 0, 0
episodes = 10
total_reward = 0
for _ in range(episodes):
    state = env.reset()
    state = encode(state)
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        state = encode(state)
        if reward == -10:
            penalties += 1
        
        epochs += 1
        total_reward += reward
#         env.render()
#         time.sleep(1/60)
    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")
print(f"Average reward per episode: {total_reward / episodes}")